In [ ]:
import housing_scraper 
import importlib 
importlib.reload(housing_scraper)
from housing_scraper import TokyoHousingScraper

import pandas as pd
import numpy as np
import re, sqlite3, prettytable 
prettytable.DEFAULT = 'DEFAULT'

# Load SQL magic extension to run SQL queries directly in notebook cells
%load_ext sql

## Housing Data Collection & Loading
>In this section, we initialize the `TokyoHousingScraper` to:  
>1. **Collect raw HTML** from *SUUMO.jp*.  
>2. **Transform the HTML** into structured, readable data and housing metrics.  
>3. **Store the processed data** in an SQLite database.

In [39]:
# Path to the SQLite database where scraped housing data will be stored
db = 'tokyo_housing.db'

# Base URL of Suumo (Japanese housing site)
base_url = 'https://suumo.jp/'

# URLs for initial listings pages 
starting_url = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&pc=50&smk=&po1=25&po2=99&shkr1=03&shkr2=03&shkr3=03&shkr4=03&rn=0025&ek=002506940&rn=0350&ek=035017990&ek=035026830&ek=035001440&rn=0070&ek=007026830&ek=007006960&ra=013&ae=00251&ae=03501&cb=0.0&ct=9999999&md=01&md=02&md=03&md=04&md=05&md=06&md=07&md=08&md=09&md=10&md=11&md=12&md=13&et=9999999&mb=0&mt=9999999&cn=9999999&fw2='

In [40]:
# Initialize scraper
scraper = TokyoHousingScraper(db, base_url, starting_url)

# Scrape housing listings
scraper.scrape_listings()

# Parse listing details and save dataset to SQLite
scraper.build_housing_dataset()

2764 properties were successfully gathered!


## Extracting & Engineering Housing Metrics 
>- Connect to local SQLite database `db` containing listing information and housing metrics.
>- Initialize SQL Magic (`%sql`) to run queries directly from the notebook. 

In [41]:
# Connect to SQLite database for querying listings 
conn = sqlite3.connect(db)
cursor = conn.cursor()

# Initialize SQL Magic with database connection
%sql sqlite:///tokyo_housing.db

### Create SQL View

>**Step 1: Standardize core listing fields**  
>- `img`, `title`, `address`: Basic identifiers  
>- `rent`, `management_fee`, `deposit`, `key_money`: Convert to numeric values  
>- `floor`: Convert floor labels to integers  
>- `floor_plan`: Normalize labels (e.g., 'ワンルーム' → '1R')  
>- `area`: Convert to numeric (square meters)  
>- `building_age`: Extract age in years  
>- `building_size`: Standardize number of floors  
>- `stations`, `nearest_station`, `distance_to_nearest_station`, `avg_distance_to_stations`: Station-related features  
>
>**Step 2: Handle missing or invalid values**  
>- Replace 0 or invalid values in `management_fee`, `deposit`, `key_money` with NULL  
>
>**Step 3: Feature engineering**  
>- `avg_rent_by_station`: Average rent per nearest station  
>- `avg_rent_by_floor_plan`: Average rent per floor plan  
>- `count_listings_per_station`: Number of listings per station (nearest)
>- `count_listings_per_floor_plan`: Number of listings per floor plan (e.g. `1DK`, `2LDK`) 
>
>**Step 4: Build final view**  
>- Combine standardized fields and engineered features into `FEATURED_LISTINGS`  
>- Output all listings in `TOKYO_HOUSING` view

In [ ]:
%%sql 
-- Remove the view if it already exists
DROP VIEW IF EXISTS TOKYO_HOUSING;

-- Create a cleaned + feature-engineered housing view
CREATE VIEW TOKYO_HOUSING AS

-- Deduplicate listings that appear multiple times due to scraping artifacts.
-- Listings are considered duplicates if they share the same title, address,
-- rent, floor plan, and floor.

-- ROW_NUMBER() is used to retain a single representative row per duplicate group.
-- Ordering by `url` provides a stable and unique tie-breaker
WITH DEDUPLICATED_LISTINGS AS (
    SELECT * 
    FROM  (
        SELECT 
            *,
            ROW_NUMBER() OVER (
                PARTITION BY title, floor, floor_plan, area, rent
                ORDER BY url
                ) 
                AS rn
        FROM HOUSING_DATA_RAW
    )
    WHERE rn = 1
),

STANDARDIZED_LISTINGS AS (
    SELECT 
        url, title, address, 
        
        CAST(REPLACE(rent, '万円', '') AS FLOAT) * 10000 AS rent,
        CAST(REPLACE(management_fee, '円', '') AS FLOAT) AS management_fee,
        CAST(REPLACE(deposit, '万円', '') AS FLOAT) * 10000 AS deposit,
        CAST(REPLACE(key_money, '万円', '') AS FLOAT) * 10000 AS key_money,
        
        RTRIM(floor, '階') AS floor,
        
        CASE
            WHEN floor_plan = 'ワンルーム' THEN '1R'
            ELSE floor_plan
        END AS floor_plan,
        
        CAST(REPLACE(area, 'm2', '') AS FLOAT) AS area,
        
        CASE 
            WHEN building_age LIKE '%新築%' THEN 0
            WHEN building_age LIKE '%以上'
                THEN CAST(REPLACE(REPLACE(building_age, '築', ''), '年以上', '') AS INTEGER)
            ELSE CAST(REPLACE(REPLACE(building_age, '築', ''), '年', '') AS INTEGER)
        END AS building_age,
        
        CASE
            WHEN building_size LIKE '%平屋%' THEN '1'
            ELSE RTRIM(building_size, '階建')
        END AS building_size,
        
        stations,
        nearest_station,
        distance_to_nearest_station,
        ROUND(avg_distance_to_stations, 2) AS avg_distance_to_stations
    FROM DEDUPLICATED_LISTINGS
),

FEATURED_LISTINGS AS (
    SELECT 
        url, title, address, rent, 
        
        -- Replace 0 values with NULLs
        NULLIF(management_fee, 0.0) AS management_fee,
        NULLIF(deposit, 0.0) AS deposit,
        NULLIF(key_money, 0.0) AS key_money,
        floor, floor_plan, area, building_age,
        building_size, nearest_station,
        distance_to_nearest_station, avg_distance_to_stations,
        
        -- Average rent by station, floor plan
        ROUND(AVG(rent) 
            OVER (PARTITION BY nearest_station), 2) 
            AS avg_rent_by_station, 
        ROUND(AVG(rent)
            OVER (PARTITION BY floor_plan), 2) 
            AS avg_rent_by_floor_plan,
        
        -- Number of listings per station, floor plan
        COUNT(title)
            OVER (PARTITION BY nearest_station)
            AS count_listings_per_station,
        COUNT(title)
            OVER (PARTITION BY floor_plan) 
            AS count_listings_per_floor_plan
    FROM STANDARDIZED_LISTINGS
)

-- Final output 
SELECT * FROM FEATURED_LISTINGS

### Load SQL View Into DataFrame
>- Use `%sql` to query `TOKYO_HOUSING` and convert results to a Dataframe for further analysis.
>- Once data is in pandas, we close the database connection. 

In [ ]:
# Query the engineered SQL view into a pandas DataFrame for analysis
tokyo_housing = %sql SELECT * FROM TOKYO_HOUSING
tokyo_housing_df = tokyo_housing.DataFrame()

# Close the DB connection 
conn.close()

## Data Overview & Normalization

### Dataset Overview
> We begin by reviewing column data types, non-null counts, and summary statistics to
identify potential inconsistencies and guide downstream cleaning decisions.

In [96]:
# Display column names, non-null counts, and dtypes
tokyo_housing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4702 entries, 0 to 4701
Data columns (total 19 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   url                            4702 non-null   object 
 1   title                          4702 non-null   object 
 2   address                        4702 non-null   object 
 3   rent                           4702 non-null   float64
 4   management_fee                 4191 non-null   float64
 5   deposit                        3058 non-null   float64
 6   key_money                      2976 non-null   float64
 7   floor                          4702 non-null   object 
 8   floor_plan                     4702 non-null   object 
 9   area                           4702 non-null   float64
 10  building_age                   4702 non-null   int64  
 11  building_size                  4702 non-null   object 
 12  nearest_station                4702 non-null   o

In [97]:
# Show summary statistics for all columns (numeric + categorical)
tokyo_housing_df.describe(include = 'all')

,url,title,address,rent,management_fee,deposit,key_money,floor,floor_plan,area,building_age,building_size,nearest_station,distance_to_nearest_station,avg_distance_to_stations,avg_rent_by_station,avg_rent_by_floor_plan,count_listings_per_station,count_listings_per_floor_plan
count,4702,4702,4702,4702.000000,4191.000000,3.058000e+03,2976.000000,4702,4702,4702.000000,4702.000000,4702,4702,4702.000000,4702.000000,4702.000000,4702.00000,4702.000000,4702.000000
unique,4702,2573,89,NaN,NaN,NaN,NaN,34,22,NaN,NaN,45,24,NaN,NaN,NaN,NaN,NaN,NaN
top,https://suumo.jp/chintai/jnc_000102443278/?bc=...,Alivis新江古田,東京都新宿区北新宿４,NaN,NaN,NaN,NaN,2,1K,NaN,NaN,2,東中野駅,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,42,185,NaN,NaN,NaN,NaN,1394,1622,NaN,NaN,1085,648,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,128502.233092,8463.111429,1.406115e+05,141514.314516,NaN,NaN,31.215032,17.763292,NaN,NaN,6.387707,10.550336,128502.232393,128502.23147,420.478094,998.720119
std,NaN,NaN,NaN,70232.762231,5736.250142,9.700479e+04,94427.767655,NaN,NaN,17.279348,16.732422,NaN,NaN,2.829187,2.396287,17520.930914,59458.34219,166.273042,543.204274
min,NaN,NaN,NaN,22000.000000,1000.000000,2.200000e+04,22000.000000,NaN,NaN,6.200000,0.000000,NaN,NaN,1.000000,1.000000,55000.000000,77692.34000,1.000000,1.000000
25%,NaN,NaN,NaN,76000.000000,4000.000000,7.700000e+04,80000.000000,NaN,NaN,20.110000,3.000000,NaN,NaN,4.000000,9.330000,118283.390000,95329.65000,315.000000,359.000000
50%,NaN,NaN,NaN,112000.000000,7500.000000,1.180000e+05,114000.000000,NaN,NaN,25.840000,13.000000,NaN,NaN,6.000000,10.670000,123574.880000,95329.65000,390.000000,1071.000000
75%,NaN,NaN,NaN,159000.000000,12000.000000,1.697500e+05,165150.000000,NaN,NaN,39.330000,30.000000,NaN,NaN,9.000000,12.000000,139800.480000,168453.81000,637.000000,1622.000000


### Floor Parsing
> The `floor` variable contains a mix of single values and ranges (e.g., `"B1-2"`).  
To support analysis and modeling, floor ranges are expanded into individual observations:
>
> - Split floor ranges into their component values with the `floor_parser` helper function
> - Expand each floor into a separate row (`explode('floor')`)
> - Convert non-numeric values to `NaN` and remove them
> - Convert the resulting values to integer type
> 
> This results in one floor value per row, enabling accurate aggregation and analysis.

In [99]:
def floor_parser(floor_str):
    #Parse a floor string into numeric floor values.

    #Examples:
    # - '3'     → [3]
    # - '3-5'   → array([3, 4, 5])
    # - 'B1'    → [-1]
    # - 'B1-2'  → array([-1, 0, 1, 2])

    # Normalize and split floor ranges (e.g. "3-5")
    floor_parts = floor_str.upper().split('-')

    # Handle malformed strings like "-" or ""
    if '' in floor_parts:
        return np.nan

    # Convert basement notation (e.g. "B1" → -1)
    if floor_parts[0].startswith('B'):
        floor_parts[0] = -int(floor_parts[0][1:])

    # Single floor (no range)
    if len(floor_parts) == 1:
        return np.array(int(floor_parts[0]))

    # Floor range
    lower_floor = int(floor_parts[0])
    upper_floor = int(floor_parts[1])

    return np.arange(lower_floor, upper_floor + 1)

In [100]:
# Normalize floor values:
#  - Split floor ranges (e.g., "B1-2" → [-1, 0, 1, 2])
#  - Expand to one floor per row
#  - Convert non-numeric values to NaN and drop them
tokyo_housing_df = (
    tokyo_housing_df
        .assign(
            floor = lambda df: df['floor'].apply(floor_parser)
        )
        .explode('floor')
        .dropna(subset = ['floor'])
)

# Convert floor to integer type
tokyo_housing_df['floor'] = tokyo_housing_df['floor'].astype('int64')

### Building Size Normalization
>The `building_size` column includes semi-structured text (e.g., `"地下1地上3"`),
representing underground and above-ground floors.
>To standardize this feature:
> - All numeric components are extracted using **regular expressions**
> - Values are summed to compute total building height
>
>The resulting feature represents **total building size** as a single numeric value.

In [104]:
# Parse total building size by extracting numeric components
#    from semi-structured strings (e.g., "地下1地上3" → 1 + 3 = 4)
tokyo_housing_df['building_size'] = (
    tokyo_housing_df['building_size']
        .map(
            lambda x: sum(map(int, re.findall(r'\d+', x)))
        )
)

In [106]:
# Save cleaned DataFrame to CSV file with UTF-8 encoding for Excel capability
tokyo_housing_df.to_csv('tokyo_housing.csv', index = False, encoding = 'utf-8-sig')